In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import pandas as pd
import json

def fetch_data(url, offset=0, limit=2000):
    """
    Fetch data from the specified URL of a Web API with pagination.
    Handles basic error checking and prints the status of the request.
    """
    try:
        params = {'$offset': offset, '$limit': limit}
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises an HTTPError for bad requests (4XX or 5XX)
        return response.json()  # Converts the JSON response to a Python dictionary
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def save_data_to_csv(data, filename):
    """
    Converts a list of dictionaries (JSON data) to a Pandas DataFrame and saves it as a CSV file.
    """
    try:
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    except Exception as e:
        print(f"Failed to save data to CSV: {e}")

def main():
    # Load configuration from JSON file
    with open('config.json', 'r') as file:
        config = json.load(file)
    
    api_url = config["api_url"]
    filename = config["filename"]
    offset = config["pagination"]["offset"]
    limit = config["pagination"]["limit"]

    total_data = []
    # Example of handling pagination
    while True:
        data = fetch_data(api_url, offset, limit)
        if data:
            total_data.extend(data)
            offset += limit  # Increment offset for next batch of data
        else:
            break  # Break if no more data is returned

    if total_data:
        # Save all data to CSV file
        save_data_to_csv(total_data, filename)

if __name__ == "__main__":
    main()


Data saved to nypd_shooting.csv
